# Pseudo-stochastic optimization model
## Imports

In [ ]:
# Import packages
using MLDataUtils, CSV, DataFrames,Plots, Statistics, StatsBase
using Gurobi, JuMP
using Random 

## Implementation of the model

In [ ]:
function proba_random(pool, df, type, n, pos)
# if type is "ban"
if type == "ban"
    # transform the dataframe by removing champion name 
    # and update the possible choices of the ban_matrix with the current pool
    df_proba = df[:,2:end] .* pool
    # find the top n champions
    top_n = sortperm(df_proba, rev=true)[1:n]
    # randomly choose one of the top n champions
    champion_row = rand(top_n)
    # retrieve the champion name
    champion_name = df[champion_row,1]
    return champion_row, champion_name
# if type is "pick"
elseif type == "pick"
    # transform the dataframe by removing champion name
    # and update the possible choices of the ban_matrix with the current pool
    df_proba = df[:,2:end]  .* pool
    # find the top 5 champions
    top_n = sortperm(df_proba[:,pos],rev=true )[1:n]
    # randomly choose one of the top n champions
    champion_row = rand(top_n)
    # retrieve champion position
    champion_col = pos
    # retrieve the champion name
    champion_name = df[champion_row,1] 
    return champion_row, champion_col, champion_name
end
end

function weighted_utility(pool, df, type, n, pos,utility)
# if type is "ban"
if type == "ban"
    # transform the dataframe by removing champion name 
    # and update the possible choices of the ban_matrix with the current pool
    df_proba = df[:,2:end] .* pool
    # weighted utility 
    weighted_utility = df_proba .* utility
    # find the top n champions
    top_n = sortperm(weighted_utility, rev=true)[1:n]
    # randomly choose one of the top n champions
    champion_row = rand(top_n)
    # retrieve the champion name
    champion_name = df[champion_row,1]
    return champion_row, champion_name
# if type is "pick"
elseif type == "pick"
    # transform the dataframe by removing champion name
    # and update the possible choices of the ban_matrix with the current pool
    df_proba = df[:,2:end]  .* pool
    # weighted utility 
    weighted_utility = df_proba .* utility
    # find the top 5 champions
    top_n = sortperm(weighted_utility[:,pos],rev=true )[1:n]
    # randomly choose one of the top n champions
    champion_row = rand(top_n)
    # retrieve champion position
    champion_col = pos
    # retrieve the champion name
    champion_name = df[champion_row,1] 
    return champion_row, champion_col, champion_name
end
end

function optimal_choice(pool, utility, type, pos)

# transform the dataframe by removing champion name 
# and update the possible choices of the ban_matrix with the current pool
utility = utility .* pool

# if type is "ban"
if type == "ban"
    # find the top n champions
    opt = findmax(utility)
    # best utility
    u_opt = opt[1]
    # retrieve the champion row
    champion_row = opt[2][1]
    # retrieve the champion col
    champion_col = opt[2][2]
    return u_opt, champion_row, champion_col
# if type is "pick"
elseif type == "pick"
    # find the top n champions
    opt = findmax(utility[:,pos])
    # best utility
    u_opt = opt[1]
    # retrieve the champion row
    champion_row = opt[2][1]
    # retrieve the champion col
    champion_col = pos
    return u_opt, champion_row, champion_col
end
end

function baseline(r,model, U_B, U_R, ban_mat, pick_mat, ban_vec,df)

# Dataframe for bans
bans = DataFrame(Ban1 = String[], Ban2 = String[], Ban3 = String[], Ban4 = String[], Ban5 = String[]) # matrix of champions banned to understand order of choices
push!(bans, [" ", " ", " ", " ", " "])
push!(bans, [" ", " ", " ", " ", " "])
# Dataframe for picks
picks = DataFrame(Pick1 = String[], Pick2 = String[], Pick3 = String[], Pick4 = String[], Pick5 = String[]) # matrix of champions picked to understand order of choices
push!(picks, [" ", " ", " ", " ", " "])
push!(picks, [" ", " ", " ", " ", " "])
push!(picks, [" ", " ", " ", " ", " "])
push!(picks, [" ", " ", " ", " ", " "])

# Define the sets
tbb = [1,3,5,14,16] # blue team ban events
trb = [2,4,6,13,15] # red team ban events
tbp = [7,10,11,18,19] # blue team pick events
trp = [8,9,12,17,20]; # red team pick events

# Define decision variables
T = 20 # number of events
n = size(U_B)[1] # number of champions
m = size(U_B)[2] -1; # number of positions
global pool = ones(n,m) # matrix of champions in the pool

# Probability matrices
ban_proba = ban_mat[:,2:end]
pick_proba = pick_mat[:,2:end]

# Event vectors for picks
picks_red = [1,2,3,4,5]
picks_blue = [1,2,3,4,5]

global banned = []
# Set dummy variables
t1 = 0
t2 = 0
t3 = 0
t4 = 0

# Define the independant variables
UB = Array(U_B[:,2:end]) # blue team utility
UR = Array(U_R[:,2:end]) # red team utility

# Total utility
BU_T = 0 # total utility of blue team
RU_T = 0; # total utility of red team

for t in 1:T
    if t in tbb
        t1 += 1
        # find champion with highest utility
        fun = optimal_choice(pool,UR,"ban",0)
        # retrieve the champion row
        blue_ban = fun[2]
        # push blue_ban to banned list
        push!(banned, blue_ban)
        # calculate utility of the red ban
        BU_T += fun[1]
        # update the pool
        pool[blue_ban,:] .= 0
        # add the red ban to the bans matrix row 2
        bans[1,t1] = ban_mat[blue_ban,1]
    
    elseif t in trb
        t2 += 1
        # choose randomly the red ban from the top-5 most probable champions
        if model == "random"
            fun = proba_random(pool,ban_mat,"ban",r,0)
        elseif model == "weighted"
            fun = weighted_utility(pool,ban_mat,"ban",r,0,UB)
        end
        # fun = top_n_random(pool, ban_mat, "ban", r,0)
        # fun = weighted_utility(pool, ban_mat, "ban", r,0, UB)
        red_ban = fun[1]
        # push red_ban to banned list
        push!(banned, red_ban)
        # calculate utility of the red ban
        RU_T += maximum(UB[red_ban, :])
        # update the pool
        pool[red_ban,:] .= 0
        # add the red ban to the bans matrix row 2
        bans[2,t2] = fun[2]


    elseif t in tbp
        t3 += 1
        # best possible pick
        opt_pick_utility = 0 
        blue_pick = 0
        pos = 0 
        # picks available
        picks_available = picks_blue[picks_blue .> 0]
        for i in picks_available 
            fun = optimal_choice(pool,UB,"pick",i)
            if fun[1] > opt_pick_utility
                opt_pick_utility = fun[1]
                # champion row
                blue_pick = fun[2]
                # champion position
                pos = fun[3]
            end
        end
        # push blue_pick to banned list
        push!(banned, blue_pick)
        # calculate utility of the blue pick
        BU_T += opt_pick_utility
        # update the pool
        pool[blue_pick,:] .= 0
        # add the red pick to the pick matrix row 1
        picks[1,t3] = pick_mat[blue_pick,1]
        # add the position of the pick in row 2
        picks[2,t3] = names(pick_mat)[pos+1]
        # remove the position from the picks_blue vector
        picks_blue[pos] = 0


    elseif t in trp
        t4 += 1
        # picks available
        picks_available = picks_red[picks_red .> 0]
        # randomly choose a number from picks_available (i.e. the position of the pick)
        pos = rand(picks_available)
        # run function
        if model == "random"
            fun = proba_random(pool,pick_mat,"pick",r,pos)
        elseif model == "weighted"
            fun = weighted_utility(pool,pick_mat,"pick",r,pos,UR)
        end
        # fun = top_n_random(pool, pick_mat, "pick", r, pos)
        # fun = weighted_utility(pool, pick_mat, "pick", r,pos, UR)
        # choose randomly the pick from the top-5 most probable champions at that position 
        red_pick = fun[1]
        # push red_pick to banned list
        push!(banned, red_pick)
        # calculate utility of the red ban
        RU_T += UR[red_pick, pos]
        # update the pool
        pool[red_pick,:] .= 0
        # add the red pick to the picks matrix row 3
        picks[3,t4] = fun[3]
        # add the position of the pick in row 4
        picks[4,t4] = names(pick_mat)[pos+1]
        # remove the position from the picks_played vector
        picks_red[pos] = 0

    else 
        println("Error")
    end 
end
return BU_T, RU_T, bans, picks
end

In [ ]:
# import the data
df = DataFrame(CSV.read("Game_data/2022_LoL_esports_match_data_from_OraclesElixir_20221016.csv", DataFrame, pool = true));

In [ ]:
 # find all unique teams for df where league = LEC 
teams = unique(df[df.league .== "LEC", :teamname])

In [ ]:
#  for each team, play each other team once
function games(model,r)
    for i in 1:10
        for j in 1:10
            if i != j
                # number of trials 
                trials = 10000

                # define BU_T and Red_U
                BU_T = 0
                RU_T = 0

                # blue team is i 
                blue_team = teams[i]
                # red team is j
                red_team = teams[j]

                # find path of blue team utility file
                blue_utility_path = "Utility_matrices/$(blue_team).csv"
                U_B = DataFrame(CSV.read(blue_utility_path, DataFrame, pool = true))[:,3:end]
                # find path of red team utility file
                red_utility_path = "Utility_matrices/$(red_team).csv"
                U_R = DataFrame(CSV.read(red_utility_path, DataFrame, pool = true))[:,3:end]

                # find path of banning and picks matrices of red team 
                ban_mat_path = "Probabilities/$(red_team)/ban_mat.csv"
                ban_mat = DataFrame(CSV.read(ban_mat_path, DataFrame, pool = true))

                pick_mat_path = "Probabilities/$(red_team)/pick_mat.csv"
                pick_mat = DataFrame(CSV.read(pick_mat_path, DataFrame, pool = true))
                
                ban_vec_path = "Probabilities/$(red_team)/ban_vec.csv"
                ban_vec = DataFrame(CSV.read(ban_vec_path, DataFrame, pool = true))
            
                # play the game 10,000 times and average the results
                for i in 1:trials
                    BU, RU, bans, picks = baseline(r, model, U_B, U_R, ban_mat, pick_mat, ban_vec, df)
                    BU_T += BU
                    RU_T += RU
                end

                BU_T = BU_T/trials
                RU_T = RU_T/trials

                # BU_T, RU_T, bans, picks = baseline(r, model, U_B, U_R, ban_mat, pick_mat, ban_vec, df)
                # add blue_team, red_team, BU_T, RU_T to Results
                push!(Results, [blue_team, red_team, BU_T, RU_T])
            end
        end
    end
    return Results
end

In [14]:
# create empty dataframe Results with columns: Blue Team, Red Team, Blue Utility, Red Utility
Results = DataFrame(Blue_Team = String[], Red_Team = String[], Blue_Utility = Float64[], Red_Utility = Float64[])
# Set randomness level
randomness = [3,5,10]
# Set model
models = ["weighted", "random"]

# run the function for each model and each randomness level
for model in models
    for r in randomness
        Results = games(model,r)
        # export results to csv
        CSV.write("Pseudo_stochastic_results/$(model)_$(r).csv", Results)
        # empty Results
        empty!(Results)
    end
end